In [7]:
# Import necessary Libraries
import pandas as pd
import numpy as np
import os
import glob

In [8]:
# #   Utility Functions
# # 
# # Function to extract the minute and second parts of the timestamp from the filename

# from scipy.spatial.transform import Rotation as R

def get_timestamp_from_filename(file_path):
    filename = os.path.basename(file_path)
    parts = filename.split('_')
    hr = int(parts[-5])
    minute = int(parts[-4])
    second = int(parts[-3].split('.')[0])  # Removing the ".csv" part
    return (hr, minute, second)

# def eul_2_quat(rpy):
#     rotation = R.from_rotvec(rpy)
#     return rotation.as_quat()

# def quat_2_eul(quat):
#     rotation = R.from_quat(quat)
#     return rotation.as_euler('xyz', degrees=False)

In [16]:
# Read the CSV Files - create a dictionary with the data

csv_folder = "lift_data/data"
data = {}

csv_files = glob.glob(os.path.join(csv_folder, '*.csv'))
sorted_file_paths = sorted(csv_files, key=get_timestamp_from_filename)
# Sort the CSV files based on name
# print(sorted_file_paths)
for n, csv_file in enumerate(sorted_file_paths):


    print(f"Reading CSV File: {csv_file}")
    timestamps = pd.read_csv(csv_file, usecols=[0], engine="python")
    joint_angles = pd.read_csv(csv_file, usecols=range(1, 7), engine="python").astype(np.float64)

    gripper_pos = np.zeros((len(joint_angles), 1))
    # gripper_pos = pd.read_csv(csv_file, usecols=[7], engine="python").astype(np.float64)
    gripper_pos = np.array(gripper_pos)
    

    ee_pos_xyz = pd.read_csv(csv_file, usecols=range(7, 10), engine="python").astype(np.float64)
    # convert into meters
    ee_pos_xyz = ee_pos_xyz/1000
    
    # ee_pos_xyz = ee_pos_xyz
    # ee_pos_rpy = pd.read_csv(csv_file, usecols=range(11, 14), engine="python").astype(np.float64)

    # Create a nested dictionary for each demonstration
    data[f'demo_{n}'] = {
        'timestamps': np.array(timestamps)[::2],
        'joint_angles': np.array(joint_angles)[::2],
        'gripper_pos': np.array(gripper_pos)[::2],
        'ee_pos_xyz': np.array(ee_pos_xyz)[::2],
        # 'timestamps': np.array(timestamps),
        # 'joint_angles': np.array(joint_angles),
        # 'gripper_pos': np.array(gripper_pos),
        # 'ee_pos_xyz': np.array(ee_pos_xyz),
        # convert rpy to quaternion
        # 'ee_pos_quat': np.array(eul_2_quat(np.array(ee_pos_rpy)[::2])),
    }
    print(f"length of demo: {len(data[f'demo_{n}']['gripper_pos'])}")

    # if n == 9:      # Only 10 trajectory
    #     break

print("----------------------------")
print(f"Total number of demos: {len(data)}")
print("----------------------------")
print(f"data: {data.keys()}")



Reading CSV File: lift_data/data/2024_10_31_17_11_15_ep_0.csv
length of demo: 47
Reading CSV File: lift_data/data/2024_10_31_17_17_41_ep_1.csv
length of demo: 28
Reading CSV File: lift_data/data/2024_10_31_17_22_41_ep_3.csv
length of demo: 35
Reading CSV File: lift_data/data/2024_10_31_17_23_32_ep_4.csv
length of demo: 38
Reading CSV File: lift_data/data/2024_10_31_17_24_13_ep_2.csv
length of demo: 43
Reading CSV File: lift_data/data/2024_10_31_17_24_50_ep_5.csv
length of demo: 37
Reading CSV File: lift_data/data/2024_10_31_17_26_11_ep_6.csv
length of demo: 43
Reading CSV File: lift_data/data/2024_10_31_17_26_50_ep_7.csv
length of demo: 51
Reading CSV File: lift_data/data/2024_10_31_17_27_45_ep_8.csv
length of demo: 41
Reading CSV File: lift_data/data/2024_10_31_17_28_25_ep_9.csv
length of demo: 42
Reading CSV File: lift_data/data/2024_10_31_17_29_24_ep_10.csv
length of demo: 31
Reading CSV File: lift_data/data/2024_10_31_17_31_21_ep_11.csv
length of demo: 34
Reading CSV File: lift_dat

In [17]:
# Read Images [cam0 - Env Camera] folder - Create a dictionary with the images

import cv2

images = ""

image_folders = glob.glob(os.path.join("lift_data/images", '*'))
# sorted_image_folders = sorted(image_folders, key=get_timestamp_from_filename)
sorted_image_folders = image_folders

# Sort the CSV files based on name
for n, image_folder in enumerate(sorted_image_folders):
    print(f"Reading Image Folder: {image_folder}")
    image_files = glob.glob(os.path.join(image_folder, '*.png'))
    
    # get number of images
    num_images = len(image_files)

    # print(sorted_image_files)
    # get all image files as numpy array
    cv_images=[]
    orig_images = []
    for i in range(0, num_images, 2):
        cv_image = cv2.imread(f"{image_folder}/frame_{i}.png")
        
        orig_images.append(cv_image)
        cv_image = cv2.resize(cv_image, (96, 96))
        # Transpose the image to (3, 320, 320)
        cv_image = np.transpose(cv_image, (2, 0, 1))
        cv_images.append(cv_image)

    print(f"Number of images: {len(cv_images)}")
    # Create a nested dictionary for each demonstration
    data[f'demo_{n}']['cam0'] = np.array(cv_images)
    data[f"demo_{n}"]['cam0_orig'] = np.array(orig_images)

    # if n == 9:      # Only 10 datasets
    #     break


print("----------------------------------------")
print(f"Total number of images folders: {len(sorted_image_folders)}")
print("----------------------------------------")


Reading Image Folder: lift_data/images/2024_10_31_17_26_44_ep_7
Number of images: 51
Reading Image Folder: lift_data/images/2024_10_31_17_24_06_ep_2
Number of images: 44
Reading Image Folder: lift_data/images/2024_10_31_17_24_45_ep_5
Number of images: 38
Reading Image Folder: lift_data/images/2024_10_31_17_41_40_ep_20
Number of images: 40
Reading Image Folder: lift_data/images/2024_10_31_17_23_22_ep_4
Number of images: 40
Reading Image Folder: lift_data/images/2024_10_31_17_33_36_ep_12
Number of images: 40
Reading Image Folder: lift_data/images/2024_10_31_17_36_26_ep_16
Number of images: 37
Reading Image Folder: lift_data/images/2024_10_31_17_46_18_ep_24
Number of images: 32
Reading Image Folder: lift_data/images/2024_10_31_17_22_35_ep_3
Number of images: 32
Reading Image Folder: lift_data/images/2024_10_31_17_35_02_ep_14
Number of images: 40
Reading Image Folder: lift_data/images/2024_10_31_17_26_05_ep_6
Number of images: 38
Reading Image Folder: lift_data/images/2024_10_31_17_35_40_e

In [18]:
# # Read Images [cam1 - In-Hand Camera] folder - Create a dictionary with the images

# import cv2

# images = ""

# image_folders = glob.glob(os.path.join("data/ButtonPush/cam1", '*'))
# sorted_image_folders = sorted(image_folders, key=get_timestamp_from_filename)
# # Sort the CSV files based on name
# for n, image_folder in enumerate(sorted_image_folders):
#     print(f"Reading Image Folder: {image_folder}")
#     image_files = glob.glob(os.path.join(image_folder, '*.png'))
    
#     # Sort the image files based on name
#     sorted_image_files = sorted(image_files, key=get_timestamp_from_filename)

#     # print(sorted_image_files)
#     # get all image files as numpy array
#     cv_images=[]
#     orig_images = []
#     for i in range(0, len(sorted_image_files), 1):
#         cv_image = cv2.imread(sorted_image_files[i])
#         orig_images.append(cv_image)
#         # cv_image = cv2.rotate(cv_image, cv2.ROTATE_90_CLOCKWISE)       # Temporary Edit
#         cv_image = cv2.resize(cv_image, (96, 96))
#         # Transpose the image to (3, 320, 320)
#         cv_image = np.transpose(cv_image, (2, 0, 1))
#         cv_images.append(cv_image)

#     print(f"Number of images: {len(cv_images)}")
#     # Create a nested dictionary for each demonstration
#     data[f'demo_{n}']['cam1'] = np.array(cv_images)
#     data[f'demo_{n}']['cam1_orig'] = np.array(orig_images)


#     # if n == 9:      # Only 10 datasets
#         # break



# print("----------------------------------------")
# print(f"Total number of images folders: {len(sorted_image_folders)}")
# print("----------------------------------------")


In [19]:
# # Read Images [cam1 - In-Hand Camera] folder - Create a dictionary with the images

# import cv2

# images = ""

# image_folders = glob.glob(os.path.join("data/ButtonPush/cam2", '*'))
# sorted_image_folders = sorted(image_folders, key=get_timestamp_from_filename)
# # Sort the CSV files based on name
# for n, image_folder in enumerate(sorted_image_folders):
#     print(f"Reading Image Folder: {image_folder}")
#     image_files = glob.glob(os.path.join(image_folder, '*.png'))
    
#     # Sort the image files based on name
#     sorted_image_files = sorted(image_files, key=get_timestamp_from_filename)

#     # print(sorted_image_files)
#     # get all image files as numpy array
#     cv_images=[]
#     orig_images = []
#     for i in range(0, len(sorted_image_files), 1):
#         cv_image = cv2.imread(sorted_image_files[i])
#         orig_images.append(cv_image)
#         # cv_image = cv2.rotate(cv_image, cv2.ROTATE_90_CLOCKWISE)       # Temporary Edit
#         cv_image = cv2.resize(cv_image, (96, 96))
#         # Transpose the image to (3, 320, 320)
#         cv_image = np.transpose(cv_image, (2, 0, 1))
#         cv_images.append(cv_image)

#     print(f"Number of images: {len(cv_images)}")
#     # Create a nested dictionary for each demonstration
#     data[f'demo_{n}']['cam2'] = np.array(cv_images)
#     data[f'demo_{n}']['cam2_orig'] = np.array(orig_images)


#     # if n == 9:      # Only 10 datasets
#         # break



# print("----------------------------------------")
# print(f"Total number of images folders: {len(sorted_image_folders)}")
# print("----------------------------------------")


In [20]:
# Compare the length of csv files to the number of images in the folder

for demo in data:
    # print(f"Data {demo}: {len(data[demo]['ee_pos_xyz'])} timestamps, cam0: {len(data[demo]['cam0'])}, cam1: {len(data[demo]['cam1'])}, cam2: {len(data[demo]['cam2'])}",)
    print(f"Data {demo}: {len(data[demo]['ee_pos_xyz'])} timestamps, cam0: {len(data[demo]['cam0'])}")
    print("i")
    

Data demo_0: 47 timestamps, cam0: 51
i
Data demo_1: 28 timestamps, cam0: 44
i
Data demo_2: 35 timestamps, cam0: 38
i
Data demo_3: 38 timestamps, cam0: 40
i
Data demo_4: 43 timestamps, cam0: 40
i
Data demo_5: 37 timestamps, cam0: 40
i
Data demo_6: 43 timestamps, cam0: 37
i
Data demo_7: 51 timestamps, cam0: 32
i
Data demo_8: 41 timestamps, cam0: 32
i
Data demo_9: 42 timestamps, cam0: 40
i
Data demo_10: 31 timestamps, cam0: 38
i
Data demo_11: 34 timestamps, cam0: 44
i
Data demo_12: 43 timestamps, cam0: 43
i
Data demo_13: 38 timestamps, cam0: 41
i
Data demo_14: 41 timestamps, cam0: 34
i
Data demo_15: 42 timestamps, cam0: 43
i
Data demo_16: 35 timestamps, cam0: 30
i
Data demo_17: 34 timestamps, cam0: 35
i
Data demo_18: 36 timestamps, cam0: 38
i
Data demo_19: 42 timestamps, cam0: 31
i
Data demo_20: 40 timestamps, cam0: 25
i
Data demo_21: 37 timestamps, cam0: 33
i
Data demo_22: 32 timestamps, cam0: 41
i
Data demo_23: 34 timestamps, cam0: 37
i
Data demo_24: 40 timestamps, cam0: 36
i


In [21]:
# Match the lengths of timestamps and images

import numpy as np

def match_lengths(data):
    for demo in data:
        num_timestamps = len(data[demo]['ee_pos_xyz'])
        cam0 = len(data[demo]['cam0'])
        # cam1 = len(data[demo]['cam1'])
        # cam2 = len(data[demo]['cam2'])

        # Find the minimum length among timestamps and cameras
        min_len = min(num_timestamps, cam0)

        # Truncate data to match the smallest length
        data[demo]['ee_pos_xyz'] = data[demo]['ee_pos_xyz'][:min_len]
        data[demo]['cam0'] = data[demo]['cam0'][:min_len]
        # data[demo]['cam1'] = data[demo]['cam1'][:min_len]
        # data[demo]['cam2'] = data[demo]['cam2'][:min_len]
        data[demo]['gripper_pos'] = data[demo]['gripper_pos'][:min_len]
        data[demo]['cam0_orig'] = data[demo]['cam0_orig'][:min_len]
        # data[demo]['cam1_orig'] = data[demo]['cam1_orig'][:min_len]
        # data[demo]['cam2_orig'] = data[demo]['cam2_orig'][:min_len]

        # Debug prints to verify the matching
        # print(f"After matching: Data {demo}: {len(data[demo]['ee_pos_xyz'])} timestamps, cam0: {len(data[demo]['cam0'])}, cam1: {len(data[demo]['cam1'])}, cam2: {len(data[demo]['cam2'])}")
        print(f"After matching: Data {demo}: {len(data[demo]['ee_pos_xyz'])} timestamps, cam0: {len(data[demo]['cam0'])}")

# Example usage
match_lengths(data)


After matching: Data demo_0: 47 timestamps, cam0: 47
After matching: Data demo_1: 28 timestamps, cam0: 28
After matching: Data demo_2: 35 timestamps, cam0: 35
After matching: Data demo_3: 38 timestamps, cam0: 38
After matching: Data demo_4: 40 timestamps, cam0: 40
After matching: Data demo_5: 37 timestamps, cam0: 37
After matching: Data demo_6: 37 timestamps, cam0: 37
After matching: Data demo_7: 32 timestamps, cam0: 32
After matching: Data demo_8: 32 timestamps, cam0: 32
After matching: Data demo_9: 40 timestamps, cam0: 40
After matching: Data demo_10: 31 timestamps, cam0: 31
After matching: Data demo_11: 34 timestamps, cam0: 34
After matching: Data demo_12: 43 timestamps, cam0: 43
After matching: Data demo_13: 38 timestamps, cam0: 38
After matching: Data demo_14: 34 timestamps, cam0: 34
After matching: Data demo_15: 42 timestamps, cam0: 42
After matching: Data demo_16: 30 timestamps, cam0: 30
After matching: Data demo_17: 34 timestamps, cam0: 34
After matching: Data demo_18: 36 times

In [22]:
# # # Preprocess the Gripper Position

# # indicies = np.array([44, 44, 43, 36, 44, 40, 34, 42, 48, 40])
# # print(indicies)

# # for n, i in enumerate(data.keys()):
# #     gripper_pos = data[i]['gripper_pos']

# #     # -------------- Change gripper pos at corresponding index --------------------
# #     gripper_pos[:indicies[n]] = 0.0
# #     gripper_pos[indicies[n]:] = 0.5

# #     data[i]['gripper_pos'] = gripper_pos

# # print("Gripper Poses Updated")


# ## Approximate gripper position - 0.6

# for demo in data.keys():
#     gripper_pos = data[demo]['gripper_pos']
#     gripper_pos_new = np.where(gripper_pos > 0.1, 0.6, 0.0)
#     data[demo]['gripper_pos'] = gripper_pos_new 
    
# print("--------------------------------------")
# print("Gripper Pos set to 0.0 or 0.6")
# print("--------------------------------------")

In [23]:
# Metaworld Dataset - HDF5
# 
# Actions - Difference (next - current)


for demo in data.keys():
    demo_data = data[demo]
    timestamps = demo_data['timestamps']
    ee_pos_xyz = demo_data['ee_pos_xyz']
    # ee_pos_quat = demo_data['ee_pos_quat']
    gripper_pos = demo_data['gripper_pos']
    print(f"Demo: {demo}, Gripper Pos: {gripper_pos.shape}")
    cam0 = demo_data['cam0']
    # cam1 = demo_data['cam1']

    # ee_pos = np.concatenate((ee_pos_xyz, ee_pos_quat), axis=1)
    # actions = ee_pos[1:] - ee_pos[:-1]
    actions = ee_pos_xyz[1:] - ee_pos_xyz[:-1]

    # actions concatenate gripper_pos
    actions = np.concatenate((actions, gripper_pos[:-1]), axis=1)
    data[demo]["actions"] = actions

    print(f"Actions Added: Dimension: {data[demo]['actions'].shape}")


Demo: demo_0, Gripper Pos: (47, 1)
Actions Added: Dimension: (46, 4)
Demo: demo_1, Gripper Pos: (28, 1)
Actions Added: Dimension: (27, 4)
Demo: demo_2, Gripper Pos: (35, 1)
Actions Added: Dimension: (34, 4)
Demo: demo_3, Gripper Pos: (38, 1)
Actions Added: Dimension: (37, 4)
Demo: demo_4, Gripper Pos: (40, 1)
Actions Added: Dimension: (39, 4)
Demo: demo_5, Gripper Pos: (37, 1)
Actions Added: Dimension: (36, 4)
Demo: demo_6, Gripper Pos: (37, 1)
Actions Added: Dimension: (36, 4)
Demo: demo_7, Gripper Pos: (32, 1)
Actions Added: Dimension: (31, 4)
Demo: demo_8, Gripper Pos: (32, 1)
Actions Added: Dimension: (31, 4)
Demo: demo_9, Gripper Pos: (40, 1)
Actions Added: Dimension: (39, 4)
Demo: demo_10, Gripper Pos: (31, 1)
Actions Added: Dimension: (30, 4)
Demo: demo_11, Gripper Pos: (34, 1)
Actions Added: Dimension: (33, 4)
Demo: demo_12, Gripper Pos: (43, 1)
Actions Added: Dimension: (42, 4)
Demo: demo_13, Gripper Pos: (38, 1)
Actions Added: Dimension: (37, 4)
Demo: demo_14, Gripper Pos: (3

In [11]:
# # check the actions
# # Concatenate gripper_pos
# first_pos = data['demo_0']['ee_pos_xyz'][0]
# # gripper_pos = d[0]
# first_pos = np.concatenate((first_pos, [0]), axis=0)

# print(f"first_pos: {first_pos}")

# # Add actions
# traj = []
# check_actions = data["demo_0"]["actions"]
# for i in check_actions:
#     first_pos[:3]+=i[:3]
#     first_pos[3:] = i[3:]
#     print(first_pos)
# # print(traj)

In [12]:
# # initialize function for generating sketch images
# import cv2
# import numpy as np
# import h5py

# def find_green_mark(image):
#     hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     lower_green = np.array([50, 100, 100])
#     upper_green = np.array([70, 255, 255])
#     mask = cv2.inRange(hsv, lower_green, upper_green)
#     contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     if contours:
#         largest_contour = max(contours, key=cv2.contourArea)
#         M = cv2.moments(largest_contour)
#         if M["m00"] != 0:
#             cx = int(M["m10"] / M["m00"])
#             cy = int(M["m01"] / M["m00"])
#             return cx, cy
#     return None

# def process_images(images):
#     trajectories = []
#     for image_data in images:
#         image = np.array(image_data)
#         if image is not None:
#             position = find_green_mark(image)
#             if position:
#                 trajectories.append(position)
#     return trajectories

# def create_blank_canvas(image):
#     height, width, channels = image.shape
#     return np.zeros((height, width, channels), dtype=np.uint8)

# def draw_trajectory_on_image(trajectories, image):
#     for i, (x, y) in enumerate(trajectories):
#         color = (255, 255, 0)
#         cv2.circle(image, (x, y), 2, color, -1)    
#         if i == 0:
#             color = (0, 255, 0)
#             cv2.circle(image, (x, y), 5, color, -1)
#         elif i == len(trajectories) - 1:
#             color = (255, 0, 0)
#             cv2.circle(image, (x, y), 5, color, -1)
#     return image



# def generate_sketch_image(camera_images):
#     trajectories = process_images(camera_images)
#     first_image = camera_images[0]
#     final_image = draw_trajectory_on_image(trajectories, first_image)
#     blank_canvas = create_blank_canvas(first_image)
#     sketch_image = draw_trajectory_on_image(trajectories, blank_canvas)

#     return final_image, sketch_image

In [16]:
# # Add sketch images
# for demo in data.keys():
#     # cam1_images = []
#     # for image in data[demo]["cam1"]:
#     #     cam1_images.append(np.transpose(image, (1, 2, 0)))
#     #     print(cam1_images[0].shape)
#     # cam2_images = []
#     # for image in data[demo]["cam2"]:
#     #     cam2_images.append(np.transpose(image, (1, 2, 0)))

#     cam1_images = data[demo]["cam1_orig"][:]
#     cam2_images = data[demo]["cam2_orig"][:]
#     rgb_sketch1, blank_sketch1 = generate_sketch_image(cam1_images)
#     rgb_sketch2, blank_sketch2 = generate_sketch_image(cam2_images)

#     data[demo]["cam1_blank_sketch"] = np.transpose(blank_sketch1, (2, 0, 1))
#     data[demo]["cam1_rgb_sketch"] = np.transpose(rgb_sketch1, (2, 0, 1))

#     data[demo]["cam2_blank_sketch"] = np.transpose(blank_sketch2, (2, 0, 1))
#     data[demo]["cam2_rgb_sketch"] = np.transpose(rgb_sketch2, (2, 0, 1))

In [24]:
# Lets build HDF5 - Metaworld Dataset


import h5py

with h5py.File('data_light_50_half.hdf5', 'w') as f:


    data_group = f.create_group("data")
    


    for demo in data.keys():

        demo_group = data_group.create_group(demo)

        # Create action
        demo_group.create_dataset("actions", data=data[demo]["actions"])


        # Create dones
        dones = np.zeros(len(data[demo]["actions"]))
        dones[-1] = 1
        demo_group.create_dataset("dones", data=dones)


        # Create obs group
        obs_group = demo_group.create_group("obs")
        # Create eye_in_hand_image
        obs_group.create_dataset("eye_in_hand_image", data=data[demo]["cam0"][:-1])
        # obs_group.create_dataset("corner_image", data=data[demo]["cam1"][:-1])
        # obs_group.create_dataset("corner2_image", data=data[demo]["cam2"][:-1])

        # obs_group.create_dataset("corner_rgb_sketch", data=data[demo]["cam1_rgb_sketch"])
        # obs_group.create_dataset("corner_blank_sketch", data=data[demo]["cam1_blank_sketch"])


        # obs_group.create_dataset("corner2_rgb_sketch", data=data[demo]["cam2_rgb_sketch"])
        # obs_group.create_dataset("corner2_blank_sketch", data=data[demo]["cam2_blank_sketch"])
        # Create prop
        prop = np.concatenate((data[demo]["ee_pos_xyz"], data[demo]["gripper_pos"]), axis=1)
        obs_group.create_dataset("prop", data=prop[:-1])


        # Create rewards
        rewards = np.zeros(len(data[demo]["actions"]))
        rewards[-1] = 1
        demo_group.create_dataset("rewards", data=rewards)


        # Create states
        demo_group.create_dataset("states", data=prop[:-1])

print("----------------------------")
print("Dataset - hdf5 Generated.. Wohoooo!")
print("----------------------------")
# print file name from above




----------------------------
Dataset - hdf5 Generated.. Wohoooo!
----------------------------


In [13]:
# Lets build HDF5 - Metaworld Dataset


import h5py

with h5py.File('data_rightleft_v2.hdf5', 'w') as f:
    untill_n = 10

    data_group = f.create_group("data")
    
    listiwant = ['demo_0', 'demo_4', 'demo_5', 'demo_8', 'demo_9', 'demo_12']

    for demo in listiwant:

        demo_group = data_group.create_group(demo)

        # Create action
        demo_group.create_dataset("actions", data=data[demo]["actions"][:untill_n])


        # Create dones
        dones = np.zeros(len(data[demo]["actions"]))
        dones[untill_n-1] = 1
        demo_group.create_dataset("dones", data=dones[:untill_n])


        # Create obs group
        obs_group = demo_group.create_group("obs")
        # Create eye_in_hand_image
        obs_group.create_dataset("eye_in_hand_image", data=data[demo]["cam0"][:-1][:untill_n])
        # obs_group.create_dataset("corner_image", data=data[demo]["cam1"][:-1])
        # obs_group.create_dataset("corner2_image", data=data[demo]["cam2"][:-1])

        # obs_group.create_dataset("corner_rgb_sketch", data=data[demo]["cam1_rgb_sketch"])
        # obs_group.create_dataset("corner_blank_sketch", data=data[demo]["cam1_blank_sketch"])


        # obs_group.create_dataset("corner2_rgb_sketch", data=data[demo]["cam2_rgb_sketch"])
        # obs_group.create_dataset("corner2_blank_sketch", data=data[demo]["cam2_blank_sketch"])
        # Create prop
        prop = np.concatenate((data[demo]["ee_pos_xyz"], data[demo]["gripper_pos"]), axis=1)
        obs_group.create_dataset("prop", data=prop[:-1][:untill_n])


        # Create rewards
        rewards = np.zeros(len(data[demo]["actions"]))
        rewards[untill_n-1] = 1
        demo_group.create_dataset("rewards", data=rewards[:untill_n])


        # Create states
        demo_group.create_dataset("states", data=prop[:-1][:untill_n])

print("----------------------------")
print("Dataset for cluter visualization - hdf5 Generated.. Wohoooo!")
print("----------------------------")
# print file name from above




----------------------------
Dataset for cluter visualization - hdf5 Generated.. Wohoooo!
----------------------------


In [11]:
import h5py
import numpy as np
from torchvision import transforms
from PIL import Image

# Define augmentation transformations
def color_jitter(image):
    transform = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
    return transform(image)

def random_crop_resize(image):
    transform = transforms.RandomResizedCrop(size=(96, 96), scale=(0.8, 1.0), ratio=(0.9, 1.1))
    return transform(image)

# Apply specific combinations of augmentations
def apply_combinations(image):
    return {
        'no_aug': image,
        'color_jitter': color_jitter(image),
        'crop_resize': random_crop_resize(image),
        'color_and_crop': color_jitter(random_crop_resize(image))
    }

# Paths to original and new HDF5 files
original_hdf5_path = '../data_50_half_mm.hdf5'
augmented_hdf5_path = 'data_50_half_mm_augmented_combinations.hdf5'

# Processing and saving augmented images
with h5py.File(original_hdf5_path, 'r') as original_file, h5py.File(augmented_hdf5_path, 'w') as augmented_file:
    data_group = original_file['data']
    augmented_data_group = augmented_file.create_group("data")
    
    for demo in data_group.keys():
        demo_group = data_group[demo]
        actions = demo_group['actions'][:]
        dones = demo_group['dones'][:]
        prop = demo_group['obs/prop'][:]
        rewards = demo_group['rewards'][:]
        states = demo_group['states'][:]
        eye_in_hand_images = demo_group['obs/eye_in_hand_image'][:]

        # Reorder dimensions for each image from (C, H, W) to (H, W, C)
        eye_in_hand_images = eye_in_hand_images.transpose(0, 2, 3, 1)

        # For each image augmentation combination, create a new augmented group
        for combo_name in ['no_aug', 'color_jitter', 'crop_resize', 'color_and_crop']:
            new_demo_name = f"{demo}_{combo_name}"
            new_demo_group = augmented_data_group.create_group(new_demo_name)
            
            # Copy data that does not need augmentation
            new_demo_group.create_dataset("actions", data=actions)
            new_demo_group.create_dataset("dones", data=dones)
            obs_group = new_demo_group.create_group("obs")
            
            print(f"Processing {new_demo_name}...")
            # Apply augmentation combination to all images in the demo
            augmented_images = []
            for img in eye_in_hand_images:
                pil_image = transforms.ToPILImage()(img)
                aug_image = apply_combinations(pil_image)[combo_name]  # Choose augmentation function
                tensor_image = transforms.ToTensor()(aug_image).numpy()  # Convert back to tensor and numpy
                augmented_images.append(tensor_image)

            # Stack augmented images and reorder to (N, C, H, W)
            # augmented_images = np.stack(augmented_images).transpose(0, 2, 3, 1)
            augmented_images = np.stack(augmented_images)       # Transpose not needed already transposed

            # Save augmented images in the new demo group
            obs_group.create_dataset("eye_in_hand_image", data=augmented_images)
            obs_group.create_dataset("prop", data=prop)
            new_demo_group.create_dataset("rewards", data=rewards)
            new_demo_group.create_dataset("states", data=states)
        print("----------------------------")

print("----------------------------")
print("Dataset - HDF5 with Specific Augmented Images Generated for Each Combination.. Wohoooo!")
print("----------------------------")


Processing demo_0_no_aug...
Processing demo_0_color_jitter...
Processing demo_0_crop_resize...
Processing demo_0_color_and_crop...
----------------------------
Processing demo_1_no_aug...
Processing demo_1_color_jitter...
Processing demo_1_crop_resize...
Processing demo_1_color_and_crop...
----------------------------
Processing demo_10_no_aug...
Processing demo_10_color_jitter...
Processing demo_10_crop_resize...
Processing demo_10_color_and_crop...
----------------------------
Processing demo_11_no_aug...
Processing demo_11_color_jitter...
Processing demo_11_crop_resize...
Processing demo_11_color_and_crop...
----------------------------
Processing demo_12_no_aug...
Processing demo_12_color_jitter...
Processing demo_12_crop_resize...
Processing demo_12_color_and_crop...
----------------------------
Processing demo_13_no_aug...
Processing demo_13_color_jitter...
Processing demo_13_crop_resize...
Processing demo_13_color_and_crop...
----------------------------
Processing demo_14_no_au

In [14]:
# auguementation with demo counter


import h5py
import numpy as np
from torchvision import transforms
from PIL import Image

# Define augmentation transformations
def color_jitter(image):
    transform = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
    return transform(image)

def random_crop_resize(image):
    transform = transforms.RandomResizedCrop(size=(96, 96), scale=(0.8, 1.0), ratio=(0.9, 1.1))
    return transform(image)

# Apply specific combinations of augmentations
def apply_combinations(image):
    return {
        'no_aug': image,
        'color_jitter': color_jitter(image),
        'crop_resize': random_crop_resize(image),
        'color_and_crop': color_jitter(random_crop_resize(image))
    }

# Paths to original and new HDF5 files
original_hdf5_path = '../data_50_half.hdf5'
augmented_hdf5_path = 'data_50_half_augmented.hdf5'

# Initialize counter for unique demo names
demo_counter = 0

# Processing and saving augmented images
with h5py.File(original_hdf5_path, 'r') as original_file, h5py.File(augmented_hdf5_path, 'w') as augmented_file:
    data_group = original_file['data']
    augmented_data_group = augmented_file.create_group("data")
    
    for demo in data_group.keys():
        demo_group = data_group[demo]
        actions = demo_group['actions'][:]
        dones = demo_group['dones'][:]
        prop = demo_group['obs/prop'][:]
        rewards = demo_group['rewards'][:]
        states = demo_group['states'][:]
        eye_in_hand_images = demo_group['obs/eye_in_hand_image'][:]

        # Reorder dimensions for each image from (C, H, W) to (H, W, C)
        eye_in_hand_images = eye_in_hand_images.transpose(0, 2, 3, 1)

        # For each image augmentation combination, create a new augmented group
        for combo_name in ['no_aug', 'color_jitter', 'crop_resize', 'color_and_crop']:
            new_demo_name = f"demo_{demo_counter}"  # Use demo_counter for unique demo name
            new_demo_group = augmented_data_group.create_group(new_demo_name)
            demo_counter += 1  # Increment the counter after creating each new demo

            # Copy data that does not need augmentation
            new_demo_group.create_dataset("actions", data=actions)
            new_demo_group.create_dataset("dones", data=dones)
            obs_group = new_demo_group.create_group("obs")
            
            print(f"Processing {new_demo_name} with augmentation {combo_name}...")
            
            # Apply augmentation combination to all images in the demo
            augmented_images = []
            for img in eye_in_hand_images:
                pil_image = transforms.ToPILImage()(img)
                aug_image = apply_combinations(pil_image)[combo_name]  # Choose augmentation function
                tensor_image = transforms.ToTensor()(aug_image).numpy()  # Convert back to tensor and numpy
                augmented_images.append(tensor_image)

            # Stack augmented images (no need to transpose as already in (N, C, H, W))
            augmented_images = np.stack(augmented_images)

            # Save augmented images in the new demo group
            obs_group.create_dataset("eye_in_hand_image", data=augmented_images)
            obs_group.create_dataset("prop", data=prop)
            new_demo_group.create_dataset("rewards", data=rewards)
            new_demo_group.create_dataset("states", data=states)
        
        print("----------------------------")

print("----------------------------")
print("Dataset - HDF5 with Specific Augmented Images Generated for Each Combination.. Wohoooo!")
print("----------------------------")


Processing demo_0 with augmentation no_aug...
Processing demo_1 with augmentation color_jitter...
Processing demo_2 with augmentation crop_resize...
Processing demo_3 with augmentation color_and_crop...
----------------------------
Processing demo_4 with augmentation no_aug...
Processing demo_5 with augmentation color_jitter...
Processing demo_6 with augmentation crop_resize...
Processing demo_7 with augmentation color_and_crop...
----------------------------
Processing demo_8 with augmentation no_aug...
Processing demo_9 with augmentation color_jitter...
Processing demo_10 with augmentation crop_resize...
Processing demo_11 with augmentation color_and_crop...
----------------------------
Processing demo_12 with augmentation no_aug...
Processing demo_13 with augmentation color_jitter...
Processing demo_14 with augmentation crop_resize...
Processing demo_15 with augmentation color_and_crop...
----------------------------
Processing demo_16 with augmentation no_aug...
Processing demo_17 

In [24]:
# # Add sketch images too
# import cv2
# import numpy as np
# import h5py

# def find_green_mark(image):
#     hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     lower_green = np.array([50, 100, 100])
#     upper_green = np.array([70, 255, 255])
#     mask = cv2.inRange(hsv, lower_green, upper_green)
#     contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     if contours:
#         largest_contour = max(contours, key=cv2.contourArea)
#         M = cv2.moments(largest_contour)
#         if M["m00"] != 0:
#             cx = int(M["m10"] / M["m00"])
#             cy = int(M["m01"] / M["m00"])
#             return cx, cy
#     return None

# def process_images(images):
#     trajectories = []
#     for image_data in images:
#         image = np.array(image_data)
#         if image is not None:
#             position = find_green_mark(image)
#             if position:
#                 trajectories.append(position)
#     return trajectories

# def create_blank_canvas(image):
#     height, width, channels = image.shape
#     return np.zeros((height, width, channels), dtype=np.uint8)

# def draw_trajectory_on_image(trajectories, image):
#     for i, (x, y) in enumerate(trajectories):
#         color = (0, 0, 255)
#         if i == 0:
#             color = (0, 255, 0)
#             cv2.circle(image, (x, y), 4, color, -1)
#         elif i == len(trajectories) - 1:
#             color = (255, 0, 0)
#             cv2.circle(image, (x, y), 4, color, -1)
#         else:
#             cv2.circle(image, (x, y), 2, color, -1)
#     return image

# def process_hdf5_file(hdf5_path):
#     with h5py.File(hdf5_path, 'a') as file:
#         print(file['data'].keys())
#         for demo in file['data'].keys():
#             camera_name = ["corner2_image", "corner_image", "eye_in_hand_image"]
#             # for dataset_name, [blank_dataset, rgb_dataset] in datasets.items():
#             for cam in camera_name:
#                 # images = file[dataset_name]
#                 images = file[f"data/{demo}/obs/{cam}"][:]
#                 trajectories = process_images(images)
#                 first_image = np.array(images[0]) if images else None
#                 final_image = draw_trajectory_on_image(trajectories, first_image)
#                 blank_canvas = create_blank_canvas(first_image)
#                 sketch_image = draw_trajectory_on_image(trajectories, blank_canvas)


# # Example usage
# hdf5_path = 'data.hdf5'
# process_hdf5_file(hdf5_path)


In [144]:
# # Lets build dataset - Robosuite Dataset
# # 
# # 1. -- Ignored attributes from default dataset --
# #       obs - [object, robot0_eef_vel_ang, robot0_eef_vel_lin, robot0_gripper_qvel, 
# #               robot0_joint_pos_cos, robot0_joint_pos_sin, robot0_joint_vel_cos, 
# #               robot0_joint_vel_sin, robot0_joint_vel]
# # 
# # 2. Skipped some columns in of states

# import h5py

# with h5py.File('data/dataset_2024_7_19/data_robosuite.hdf5', 'w') as f:

#     data_group_r = f.create_group("data")

#     for demo in data.keys():
        
#         demo_group_r = data_group_r.create_group(demo)

#         # Create actions
#         demo_group_r.create_dataset("actions", data=data[demo]["actions"])

#         # Create dones
#         dones_r = np.zeros(len(data[demo]["actions"]))
#         dones_r[-1] = 1
#         demo_group_r.create_dataset("dones", data=dones_r)

#         # Create obs group
#         obs_group_r = demo_group_r.create_group("obs")
#         # Create agentview_image
#         obs_group_r.create_dataset("agentview_image", data=data[demo]["images"][:-1])
#         # Create robot0_eef_pos
#         obs_group_r.create_dataset("robot0_eef_pos", data=data[demo]["ee_pos_rpy"][:, :3][:-1])
#         # Create robot0_eef_rpy
#         obs_group_r.create_dataset("robot0_eef_rpy", data=data[demo]["ee_pos_rpy"][:, 3:][:-1])
#         # Create robot0_gripper_qpos
#         obs_group_r.create_dataset("robot0_gripper_qpos", data=data[demo]["gripper_pos"][:-1])
#         # Create robot0_joint_pos
#         obs_group_r.create_dataset("robot0_joint_pos", data=data[demo]["joint_angles"][:-1])

#         # Create rewards
#         rewards_r = np.zeros(len(data[demo]["actions"]))
#         rewards_r[-1] = 1
#         demo_group_r.create_dataset("rewards", data=rewards_r)

#         # Create states
#         # [time, joint_angles, gripper_pos, ee_pos_rpy, first_obj_pose, second_obj_pose]
#         states = np.concatenate((data[demo]["joint_angles"], data[demo]["gripper_pos"], data[demo]["ee_pos_rpy"]), axis=1)
#         demo_group_r.create_dataset("states", data=states[:-1])



In [53]:
# # Dataset - Metaworld [Bad Data - demo1 and demo6]


# # def change_gripper_pos(gripper_pos_array):
# #     indices = [46, 49, 47, 47, 49, 43, 41, 50]
# #     gripper_pos = np.zeros()



# import h5py

# with h5py.File('data/dataset_2024_7_25_2/data_processed_new.hdf5', 'w') as f:


#     data_group_b = f.create_group("data")

#     indices = [46, 49, 47, 47, 49, 43, 41, 50]


#     n = 0
#     for demo in data.keys():

#         if demo == "demo_5":
#             continue

#         demo_group_b = data_group_b.create_group(f"demo_{n}")

#         # Create action
#         demo_group_b.create_dataset("actions", data=data[demo]["actions"])

#         # Create dones
#         dones_b = np.zeros(len(data[demo]["actions"]))
#         dones_b[-1] = 1
#         demo_group_b.create_dataset("dones", data=dones_b)


#         # Create obs group
#         obs_group_b = demo_group_b.create_group("obs")
#         # Create corner2_image
#         obs_group_b.create_dataset("eye_in_hand_image", data=data[demo]["cam0"][:-1])
#         obs_group_b.create_dataset("front_image", data=data[demo]["cam1"][:-1])
#         # obs_group_b.create_dataset("eye_in_hand_image", data=data[demo]["images"][:-1])
#         # Create prop
#         # ee_pos = np.concatenate((data[demo]["ee_pos_xyz"], data[demo]["ee_pos_quat"]), axis=1)
#         prop_b = np.concatenate((data[demo]["ee_pos_xyz"], data[demo]["gripper_pos"]), axis=1)
#         obs_group_b.create_dataset("prop", data=prop_b[:-1])


#         # Create rewards
#         rewards_b = np.zeros(len(data[demo]["actions"]))
#         rewards_b[-1] = 1
#         demo_group_b.create_dataset("rewards", data=rewards_b)


#         # Create states
#         states_b = np.concatenate((data[demo]["ee_pos_xyz"], data[demo]["gripper_pos"]), axis=1)
#         demo_group_b.create_dataset("states", data=states_b[:-1])

#         n+=1

# print("----------------------------")
# print("Processed Dataset - hdf5 Generated")
# print("----------------------------")

In [146]:
# ## --- remove some points in the dataset -----------

# import h5py
# import numpy as np

# with h5py.File('data/dataset_2024_07_25/data_removed_lift.hdf5', 'w') as f:
    
#     data_group_b = f.create_group("data")

#     indices = [46, 49, 47, 47, 49, 43, 41, 50]


#     n = 0
#     for demo in data.keys():

#         if demo == "demo_1":
#             continue

#         demo_group_b = data_group_b.create_group(f"demo_{n}")

#         # Create action
#         demo_group_b.create_dataset("actions", data=data[demo]["actions"][:indices[n]+1])

#         # Create dones
#         dones_b = np.zeros(len(data[demo]["actions"][:indices[n]+1]))
#         dones_b[-1] = 1
#         demo_group_b.create_dataset("dones", data=dones_b)


#         # Create obs group
#         obs_group_b = demo_group_b.create_group("obs")
#         # Create corner2_image
#         obs_group_b.create_dataset("corner2_image", data=data[demo]["cam0"][:indices[n]+1])
#         obs_group_b.create_dataset("front_image", data=data[demo]["cam1"][:indices[n]+1])
#         # obs_group_b.create_dataset("corner2_image", data=data[demo]["images"][:-1])
#         # Create prop
#         # ee_pos = np.concatenate((data[demo]["ee_pos_xyz"], data[demo]["ee_pos_quat"]), axis=1)
#         prop_b = np.concatenate((data[demo]["ee_pos_xyz"][:indices[n]+1], data[demo]["gripper_pos"][:indices[n]+1]), axis=1)
#         obs_group_b.create_dataset("prop", data=prop_b)


#         # Create rewards
#         rewards_b = np.zeros(len(data[demo]["actions"][:indices[n]+1]))
#         rewards_b[-1] = 1
#         demo_group_b.create_dataset("rewards", data=rewards_b)


#         # Create states
#         states_b = np.concatenate((data[demo]["ee_pos_xyz"][:indices[n]+1], data[demo]["gripper_pos"][:indices[n]+1]), axis=1)
#         demo_group_b.create_dataset("states", data=states_b)

#         n+=1

# print("----------------------------")
# print("Removed Dataset - hdf5 Generated")
# print("----------------------------")

In [24]:
## Get (min, max) of x, y, z

import h5py
import numpy as np

with h5py.File('data/dataset_2024_07_25/datanew.hdf5', 'r') as f:
    
    data_group_b = f["data"]
    all_x = []
    all_y = []
    all_z = []
    for demo in data_group_b:
        states = data_group_b[demo]["states"][:]
        x = states[:,0]
        y = states[:,1]
        z = states[:,2]
        # w_ = states[:,3]
        # x_ = states[:,4]
        # y_ = states[:,5]
        # z_ = states[:,6]
        all_x.append(x)
        all_y.append(y)
        all_z.append(z)

    all_x = np.concatenate(all_x)
    all_y = np.concatenate(all_y)
    all_z = np.concatenate(all_z)
    
    print(f"min x: {np.round(np.min(all_x), 3)}")
    print(f"min y: {np.round(np.min(all_y), 3)}")
    print(f"min z: {np.round(np.min(all_z), 3)}")
    print(f"max x: {np.round(np.max(all_x), 3)}")
    print(f"max y: {np.round(np.max(all_y), 3)}")
    print(f"max z: {np.round(np.max(all_z), 3)}")

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'data/dataset_2024_07_25/datanew.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)